In [1]:
import pandas as pd
import numpy as np
import re
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [112]:
source_t = 'source/WPP2015_INT_F03_1_POPULATION_BY_AGE_ANNUAL_BOTH_SEXES.XLS'
source_m = 'source/WPP2015_INT_F03_2_POPULATION_BY_AGE_ANNUAL_MALE.XLS'
source_f = 'source/WPP2015_INT_F03_3_POPULATION_BY_AGE_ANNUAL_FEMALE.XLS'

In [118]:
def read_cleanup(source, gender):
    data_est = pd.read_excel(source, sheetname='ESTIMATES', skiprows=16, na_values='…')
    data_var = pd.read_excel(source, sheetname='MEDIUM VARIANT', skiprows=16, na_values='…')
       
    # rename/drop some columns.
    # for 80+ and 100+ groups, rename to 80plus and 100plus
    data_est = data_est.drop(['Index', 'Notes'], axis=1)
    data_var = data_var.drop(['Index', 'Notes'], axis=1)

    data_est = data_est.rename(columns={'80+': '80plus', 
                                        '100+': '100plus'})
    data_var = data_var.rename(columns={'100+': '100plus'})  # todo: no use to rename for now.
    
    
    
    # insert Gender column and rearrange the order
    col_est_1 = data_est.columns[:4]
    col_est_2 = data_est.columns[4:]
    
    col_var_1 = data_var.columns[:4]
    col_var_2 = data_var.columns[4:]
    
    cols_est = [*col_est_1, 'Gender', *col_est_2]
    cols_var = [*col_var_1, 'Gender', *col_var_2]


    data_est['Gender'] = gender
    data_var['Gender'] = gender

    return (data_est[cols_est], data_var[cols_var])

In [119]:
est_t, var_t = read_cleanup(source_t, 'both_sexes')

In [121]:
est_m, var_m = read_cleanup(source_m, 'both_sexes')

In [122]:
est_f, var_f = read_cleanup(source_f, 'both_sexes')

In [125]:
e1 = est_t.set_index([
        'Variant', 'Major area, region, country or area *', 
        'Country code', 'Reference date (as of 1 July)', 'Gender'])

In [126]:
e1.columns.name = 'age'

In [135]:
to_concat = []

for df in [est_t, var_t, est_m, var_m, est_f, var_f]:
    e = est_t.set_index([
        'Variant', 'Major area, region, country or area *', 
        'Country code', 'Reference date (as of 1 July)', 'Gender'])
    e.columns.name = 'Age'
    df_new = e.stack().reset_index().rename(columns={0: 'Population'})
    to_concat.append(df_new)

In [140]:
df_all = pd.concat(to_concat)

In [141]:
df_all = df_all.drop('Major area, region, country or area *', axis=1)
df_all = df_all.rename(columns={'Reference date (as of 1 July)': 'Year'})

df_all.columns = list(map(to_concept_id, df_all.columns))

In [142]:
df_all = df_all.sort_values(by=['country_code', 'year', 'age', 'gender'])

In [144]:
df_all.to_csv('../ddf--datapoints--population--by--country_code--year--age--gender.csv', index=False)

In [145]:
df_all.head()

,variant,country_code,year,gender,age,population
545538,Estimates,4,1950,both_sexes,0,294.58
545538,Estimates,4,1950,both_sexes,0,294.58
545538,Estimates,4,1950,both_sexes,0,294.58
545538,Estimates,4,1950,both_sexes,0,294.58
545538,Estimates,4,1950,both_sexes,0,294.58


In [146]:
e1 = e1.stack().reset_index()

In [148]:
e1 = e1.rename(columns={'Major area, region, country or area *': 'Name',
                        0: 'Population'
                       })

concs = e1.columns

e1.columns = list(map(to_concept_id, e1.columns))

In [151]:
e1 = e1.rename(columns={'reference_date_as_of_1_july': 'year'})

In [162]:
conc_df = pd.DataFrame([], columns=['concept', 'concept_type', 'name'])

In [163]:
conc_df['name'] = concs

In [165]:
conc_df['concept'] = e1.columns

In [167]:
conc_df['concept_type'] = 'string'

conc_df['concept_type'].iloc[[2, 4, 5]] = 'entity_domain'

conc_df['concept_type'].iloc[3] = 'time'

conc_df['concept_type'].iloc[6] = 'measure'

In [168]:
conc_df

,concept,concept_type,name
0,variant,string,Variant
1,name,string,Name
2,country_code,entity_domain,Country code
3,year,time,Reference date (as of 1 July)
4,gender,entity_domain,Gender
5,age,entity_domain,age
6,population,measure,Population


In [169]:
conc_df.to_csv('../ddf--concepts.csv', index=False)

In [152]:
age = df_all['age'].unique()

In [154]:
age_df = pd.DataFrame([], columns=['age', 'name'])

In [155]:
age_df['age'] = age
age_df['name'] = 'Age ' + age_df['age']

In [156]:
age_df.to_csv('../ddf--entities--age.csv', index=False)

In [159]:
gender = pd.DataFrame([], columns=['gender', 'name'])

In [160]:
gender['gender'] = ['both_sexes', 'male', 'female']
gender['name'] = ['Both sexes', 'Male', 'Female']

In [161]:
gender.to_csv('../ddf--entities--gender.csv', index=False)

In [41]:
data_est = data_est.drop(['Index', 'Notes', '80+'], axis=1)

In [ ]:
data_var = data_var.drop(['Index', 'Notes'], axis=1)

In [11]:
data_est.head()

,Index,Variant,"Major area, region, country or area *",Notes,Country code,Reference date (as of 1 July),0,1,2,3,...,91,92,93,94,95,96,97,98,99,100+
0,1,Estimates,WORLD,NaN,900,1950,78116.685,71707.672,66449.346,62228.019,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Estimates,WORLD,NaN,900,1951,81973.089,76250.302,70474.481,65672.988,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Estimates,WORLD,NaN,900,1952,85085.460,79060.671,74351.359,69205.529,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Estimates,WORLD,NaN,900,1953,87298.857,81532.381,76324.468,72491.108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Estimates,WORLD,NaN,900,1954,88613.869,83346.474,78486.514,74028.307,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
data_var.tail()

,Index,Variant,"Major area, region, country or area *",Notes,Country code,Reference date (as of 1 July),0,1,2,3,...,91,92,93,94,95,96,97,98,99,100+
20721,20722,Medium variant,Tonga,NaN,776,2096,1.968,1.969,1.965,1.962,...,0.522,0.450,0.384,0.326,0.275,0.214,0.167,0.131,0.103,0.240
20722,20723,Medium variant,Tonga,NaN,776,2097,1.957,1.957,1.961,1.959,...,0.538,0.470,0.401,0.339,0.285,0.238,0.178,0.139,0.111,0.257
20723,20724,Medium variant,Tonga,NaN,776,2098,1.944,1.947,1.947,1.953,...,0.551,0.484,0.418,0.351,0.294,0.244,0.201,0.149,0.117,0.273
20724,20725,Medium variant,Tonga,NaN,776,2099,1.930,1.933,1.938,1.940,...,0.555,0.493,0.429,0.367,0.302,0.247,0.203,0.165,0.125,0.289
20725,20726,Medium variant,Tonga,NaN,776,2100,1.911,1.917,1.925,1.928,...,0.555,0.497,0.436,0.375,0.314,0.253,0.202,0.162,0.128,0.307


In [44]:
cols_1 = data_est.columns
cols_new_1 = [*cols_1[:6], *(map(lambda x: 'Total Population aged '+ x + ' (Number)', cols_1[6:]))]

In [43]:
cols_2 = data_var.columns
cols_new_2 = [*cols_2[:6], *(map(lambda x: 'Total Population aged '+ x + ' (Number)', cols_2[6:]))]

In [45]:
cols_new_1 == cols_new_2

True

In [46]:
data_est.columns = cols_new_1
data_var.columns = cols_new_2

In [47]:
data_est.columns = data_est.columns.map(to_concept_id)

In [48]:
data_est = data_est.drop(['index', 'notes'], axis=1)

In [54]:
data_est = data_est.rename(columns={'reference_date_as_of_1_july': 'year'})

In [55]:
data_est.head()

,variant,major_area_region_country_or_area,country_code,year,total_population_aged_0_number,total_population_aged_1_number,total_population_aged_2_number,total_population_aged_3_number,total_population_aged_4_number,total_population_aged_5_number,...,total_population_aged_91_number,total_population_aged_92_number,total_population_aged_93_number,total_population_aged_94_number,total_population_aged_95_number,total_population_aged_96_number,total_population_aged_97_number,total_population_aged_98_number,total_population_aged_99_number,total_population_aged_100_number
0,Estimates,WORLD,900,1950,78116.685,71707.672,66449.346,62228.019,58930.171,56442.125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Estimates,WORLD,900,1951,81973.089,76250.302,70474.481,65672.988,61757.344,58639.277,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Estimates,WORLD,900,1952,85085.460,79060.671,74351.359,69205.529,64859.079,61248.694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Estimates,WORLD,900,1953,87298.857,81532.381,76324.468,72491.108,67974.254,64081.777,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Estimates,WORLD,900,1954,88613.869,83346.474,78486.514,74028.307,70707.145,66815.402,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
data_var.columns = data_var.columns.map(to_concept_id)
data_var = data_var.drop(['index', 'notes'], axis=1)

In [56]:
data_var = data_var.rename(columns={'reference_date_as_of_1_july': 'year'})

In [95]:
concs = pd.DataFrame([], columns=['name', 'concept'])

In [61]:
cols_new_1.remove('Index')

In [62]:
cols_new_1.remove('Notes')

In [96]:
concs['name'] = cols_new_1

In [97]:
concs['concept'] = data_est.columns

In [98]:
concs['concept_type'] = 'string'

concs['concept_type'].iloc[2] = 'entity_domain'
concs['concept_type'].iloc[3] = 'time'
concs['concept_type'].iloc[4:] = 'measure'

In [104]:
concs['unit'] = 'thousands'

concs.loc[:3, 'unit'] = np.nan

In [106]:
concs = concs.append(pd.DataFrame([
            ['unit', 'Unit', 'string', np.nan], 
            ['name', 'Name', 'string', np.nan]], columns=concs.columns))

In [107]:
concs.sort_index().head(20)

,name,concept,concept_type,unit
0,Variant,variant,string,NaN
0,unit,Unit,string,NaN
1,name,Name,string,NaN
1,"Major area, region, country or area *",major_area_region_country_or_area,string,NaN
2,Country code,country_code,entity_domain,NaN
3,Reference date (as of 1 July),year,time,NaN
4,Total Population aged 0 (Number),total_population_aged_0_number,measure,thousands
5,Total Population aged 1 (Number),total_population_aged_1_number,measure,thousands
6,Total Population aged 2 (Number),total_population_aged_2_number,measure,thousands
7,Total Population aged 3 (Number),total_population_aged_3_number,measure,thousands


In [85]:
concs.sort_index().to_csv('../ddf--concepts.csv', index=False)

In [77]:
# country

In [74]:
entity = data_est[['major_area_region_country_or_area', 'country_code']]

In [82]:
entity = entity.rename(columns={'major_area_region_country_or_area': 'name'})

In [157]:
entity.drop_duplicates().to_csv('../ddf--entities--country_code.csv', index=False)

In [87]:
len(entity.drop_duplicates())

241

In [88]:
len(data_var[['major_area_region_country_or_area', 'country_code']].drop_duplicates())

241

In [83]:
# data point

In [76]:
dp_1 = data_est.drop('major_area_region_country_or_area', axis=1)
dp_2 = data_var.drop('major_area_region_country_or_area', axis=1)

In [77]:
dp = pd.concat([dp_1, dp_2])

In [79]:
dp = dp.set_index(['country_code', 'year', 'variant'])

In [89]:
for name, df in dp.iteritems():
    path = os.path.join('../', 'ddf--datapoints--{}--by--country_code--year.csv'.format(name))
    df.reset_index().sort_values(by=['country_code', 'year']).dropna().to_csv(path, index=False)

In [170]:
create_index_file('../')

,key,value,file
0,concept,concept_type,ddf--concepts.csv
1,concept,name,ddf--concepts.csv
0,"country_code,year,gender,age",population,ddf--datapoints--population--by--country_code-...
0,age,name,ddf--entities--age.csv
0,country_code,name,ddf--entities--country_code.csv
0,gender,name,ddf--entities--gender.csv


In [168]:
from ddf_utils import ddf_reader as ddf

In [169]:
ddf.SEARCH_PATH = '/Users/semio/src/work/Gapminder/'

In [177]:
ddf.ddf_datapoint('ddf--gapminder--systema_globalis', 'crude_birth_rate_births_per_1000_population')

,geo,crude_birth_rate_births_per_1000_population,time
0,afg,48.14,1800
1,afg,48.14,1801
2,afg,48.14,1802
3,afg,48.14,1803
4,afg,48.14,1804
5,afg,48.14,1805
6,afg,48.14,1806
7,afg,48.14,1807
8,afg,48.14,1808
9,afg,48.14,1809
